In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ln_utils import handle_img

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")

In [ ]:
markers = ["s","*","o","^","v",">","D",]

# 1. Node traffic and income

drop_disabled = False

In [ ]:
drop_disabled = True

if drop_disabled:
    sim_dirs = {
        10000:"2019-09-05_14:35:13_10000sat_k6000_aNone_dropTrue",
        25000:"2019-09-05_14:35:13_25000sat_k6000_aNone_dropTrue",
        50000:"2019-09-05_14:35:13_50000sat_k6000_aNone_dropTrue",
        100000:"2019-09-05_14:35:13_100000sat_k6000_aNone_dropTrue",
        200000:"2019-09-05_14:35:13_200000sat_k6000_aNone_dropTrue",
        500000:"2019-09-05_14:35:13_500000sat_k6000_aNone_dropTrue",
    }
    root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/dropTrue/"
else:
    sim_dirs = {
        500000:"2019-09-04_14:12:21_500000sat_k5000_aNone_dropFalse",
        200000:"2019-09-04_14:12:21_200000sat_k5000_aNone_dropFalse",
        100000:"2019-09-02_17:05:18_100000sat_k5000_aNone_dropFalse",
        50000:"2019-09-03_15:28:05_50000sat_k5000_aNone_dropFalse",
        25000:"2019-09-04_14:12:21_25000sat_k5000_aNone_dropFalse",
        10000:"2019-09-04_14:12:21_10000sat_k5000_aNone_dropFalse"
    }
    root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/dropFalse/"

In [ ]:
sim_dirs = {
10000:"2019-09-07_10:23:25_10000sat_k6000",
25000:"2019-09-07_05:06:14_25000sat_k6000",
50000:"2019-09-06_22:03:19_50000sat_k6000",
100000:"2019-09-07_14:24:11_100000sat_k6000",
200000:"2019-09-07_18:37:32_200000sat_k6000",
500000:"2019-09-07_22:40:54_500000sat_k6000",
}
root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/1days_40snapshot/"

In [ ]:
node_name_replacements = {
    "rompert.com≡ƒö╡":"rompert.com",
    "03021c5f5f57322740e4":"BlueWallet",
    "≡ƒÜÇ≡ƒîæ BOLTENING.club":"BOLTENING.club"
}

In [ ]:
parts = []
for amount, folder in sim_dirs.items():
    df = pd.read_csv("%s/%s.csv" % (root_dir, folder))
    df["amount"] = amount
    parts.append(df.replace(node_name_replacements).rename({"fee":"income","num_trans":"traffic"},axis=1))
traffic_and_income_df = pd.concat(parts)

## Data Description

- income: total routing income for the given entity
- traffic: number of transactions routed through the given entity
- name: name of the entity (for most of them it is a single node BUT for LNBIG.com it is approx. 25 node)
- amount: the amount of each transaction in a given simulation

In [ ]:
traffic_and_income_df.head()

nodes = list(traffic_and_income_df.groupby("name")["income"].sum().sort_values(ascending=False).index)

nodes = ["LNBIG.com","zigzag.io","yalls.org","ln1.satoshilabs.com","BlueWallet","OpenNode","tippin.me","ACINQ","LightningPowerUsers.com","Bitrefill.com","Sagittarius A","1ML.com node ALPHA","BitMEXResearch","LightningTo.Me","fairly.cheap"]

In [ ]:
nodes = ["rompert.com","LNBIG.com","zigzag.io","yalls.org","ln1.satoshilabs.com","BlueWallet","tippin.me","ACINQ","LightningPowerUsers.com","Bitrefill.com","1ML.com node ALPHA","LightningTo.Me","fairly.cheap"]

In [ ]:
def plot_col_wrt_tx_amount(df, col, nodes, fig_size=(15,10)):
    plt.figure(figsize=fig_size)
    for i, n in enumerate(nodes):
        node_df = df[df["name"]==n]
        x, y = node_df["amount"], node_df[col]
        ls = "-" if (i+1) // len(markers) == 0 else "--" 
        plt.plot(x, y, marker=markers[i%len(markers)], markersize=10, linewidth=3, linestyle=ls, label=n)
    plt.xlabel("simulated amount per transaction (SAT)")
    plt.ylabel(col)
    plt.yscale("log")
    plt.xscale("log")

## Results

### a.) Routing income with respect to tx_amount

In [ ]:
plot_col_wrt_tx_amount(traffic_and_income_df, "income", nodes)
handle_img("router_income_drop%s" % drop_disabled, outside=True, img_dir=root_dir)

### b.) Routing traffic with respect to tx_amount

In [ ]:
plot_col_wrt_tx_amount(traffic_and_income_df, "traffic", nodes)
handle_img("router_traffic_drop%s" % drop_disabled, outside=True, img_dir=root_dir)

### c.) Routing income per transaction (calculated from routed traffic) with respect to tx_amount

In [ ]:
ipt_col = "income per transaction (SAT)"

In [ ]:
traffic_and_income_df[ipt_col] = traffic_and_income_df["income"] / traffic_and_income_df["traffic"]

In [ ]:
plot_col_wrt_tx_amount(traffic_and_income_df, ipt_col, nodes)
handle_img("node_income_per_transaction_drop%s" % drop_disabled, outside=True, img_dir=root_dir)

# 2. ROI analysis

## Load meta data from 1ML.com

In [ ]:
node_names = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_names.csv")

In [ ]:
LNBIG_data = node_names[node_names["is_lnbig"]]
len(LNBIG_data)

In [ ]:
LNBIG_record = LNBIG_data[["cap_ratio","cap_sat"]].sum()
LNBIG_record["name"] = "LNBIG.com"

In [ ]:
other_routers = [
    "02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f0c544b34c1c6f3b9a02b",#rompert.com
    "0232e20e7b68b9b673fb25f48322b151a93186bffe4550045040673797ceca43cf",#zigzag.io
    "03e50492eab4107a773141bb419e107bda3de3d55652e6e1a41225f06a0bbf2d56",#yalls.org
    "0279c22ed7a068d10dc1a38ae66d2d6461e269226c60258c021b1ddcdfe4b00bc4",#ln1.satoshilabs.com
    "03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff402de65c6eb2e1be55dc3e",#OpenNode
    "03c2abfa93eacec04721c019644584424aab2ba4dff3ac9bdab4e9c97007491dda",#tippin.me
    "0331f80652fb840239df8dc99205792bba2e559a05469915804c08420230e23c7c",#LightningPowerUsers.com
    "03021c5f5f57322740e4ee6936452add19dc7ea7ccf90635f95119ab82a62ae268",#bluewallet - 03021c5f5f57322740e4
    "028dcc199be86786818c8c32bffe9db8855c5fca98951eec99d1fa335d841605c2",#btc.lnetwork.tokyo
    "0217890e3aad8d35bc054f43acc00084b25229ecff0ab68debd82883ad65ee8266",#1ML.com node ALPHA
    "03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f",#ACINQ
    "02529db69fd2ebd3126fb66fafa234fc3544477a23d509fe93ed229bb0e92e4fb8",#Boltening.club
    "02cdf83ef8e45908b1092125d25c68dcec7751ca8d39f557775cd842e5bc127469",#tady je slushovo
    "03ee180e8ee07f1f9c9987d98b5d5decf6bad7d058bdd8be3ad97c8e0dd2cdc7ba",#Electrophorus [W_C_B]
    "03a503d8e30f2ff407096d235b5db63b4fcf3f89a653acb6f43d3fc492a7674019",#Sagittarius A
    "030c3f19d742ca294a55c00376b3b355c3c90d61c6b6b39554dbc7ac19b141c14f",#Bitrefill.com
    "03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d",#LightningTo.Me
    "0242a4ae0c5bef18048fbecf995094b74bfb0f7391418d71ed394784373f41e4f3",#CoinGate
    "03cb7983dc247f9f81a0fa2dfa3ce1c255365f7279c8dd143e086ca333df10e278",#fairly.cheap
    "031678745383bd273b4c3dbefc8ffbf4847d85c2f62d3407c0c980430b3257c403",#lightning-roulette.com
]

In [ ]:
router_node_data = node_names[node_names["pub_key"].isin(other_routers)][["name","cap_ratio","cap_sat"]].replace(node_name_replacements)

In [ ]:
router_node_data = router_node_data.append(LNBIG_record, ignore_index=True)

In [ ]:
router_node_data.sort_values("cap_ratio", ascending=False).head()

#### Almost all capacity is accounted for...

In [ ]:
router_node_data["cap_ratio"].sum()

## Calculate ROI based on simulation

- number of daily transactions: 5000
- tx_amount: 50K SAT (~5,2 USD)

In [ ]:
node_efficiency = traffic_and_income_df[traffic_and_income_df["amount"]==50000][["name","income","traffic"]].rename({"income":"daily_income","traffic":"daily_traffic"}, axis=1)

In [ ]:
router_node_data = router_node_data.merge(node_efficiency, on="name")

In [ ]:
router_node_data["annual_roi"] = router_node_data.apply(lambda x: x["daily_income"]*365.0/x["cap_sat"], axis=1)

In [ ]:
router_node_data["dupl_factor_5%_roi"] = router_node_data.apply(lambda x: (x["cap_sat"]*0.05)/(x["daily_income"]*365.0), axis=1)

In [ ]:
router_node_data["annual_roi_rank"] = router_node_data["annual_roi"].rank(ascending=False).astype("int64")
router_node_data["daily_traffic_rank"] = router_node_data["daily_traffic"].rank(ascending=False).astype("int64")
router_node_data = router_node_data.sort_values("daily_income",ascending=False).reset_index(drop=True)

In [ ]:
router_node_data

In [ ]:
router_node_data.to_csv("%s/roi_analysis_drop%s.csv" % (root_dir, drop_disabled), index=False)

In [ ]:
s = str(router_node_data.to_latex(index=False))

In [ ]:
s = s.replace("name","Entity name")
s = s.replace("cap\_ratio","Cap. ratio")
s = s.replace("cap\_sat","Entity capacity")
s = s.replace("daily\_income","Daily income")
s = s.replace("daily\_traffic","Daily traffic")
s = s.replace("annual\_roi","Annual ROI")
s = s.replace("dupl\_factor\_5\%\_roi","Duplication factor")
s = s.replace("Annual ROI\_rank","ROI rank")
s = s.replace("Daily traffic\_rank","Traffic rank")

In [ ]:
print(s)